In [10]:
from scipy.spatial.distance import euclidean
from ultralytics import YOLO
import math
import cv2
import numpy as np
from collections import defaultdict
import torch
from datetime import datetime,timedelta

In [11]:
# # Open the video file
# video_path = "/Users/adityagoyal/Desktop/ML internship/yolo/useful_footage/D9_S20250730121500_E20250730121955.mp4"
# cap = cv2.VideoCapture(video_path)

# # count the number of frames
# total_frames  = cap.get(cv2.CAP_PROP_FRAME_COUNT)
# fps = cap.get(cv2.CAP_PROP_FPS)
# seconds_of_video = total_frames / fps

# Load a pretrained YOLO11n model
model = YOLO("yolov8n.pt")

# model.track(source=video_path,show=True,persist=True,conf=0.45)

In [ ]:
results = model.train(data="/Users/adityagoyal/Desktop/ML internship/yolo/Bike Helmet Detection.v2-more-preprocessing-augmentation.yolov8 2/data_updated.yaml", epochs=100, imgsz=1000, exist_ok=True,plots=True,optimizer='Adam',patience=15)

New https://pypi.org/project/ultralytics/8.3.174 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.173 🚀 Python-3.11.13 torch-2.7.1 CPU (Apple M2 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/adityagoyal/Desktop/ML internship/yolo/Bike Helmet Detection.v2-more-preprocessing-augmentation.yolov8 2/data_updated.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1000, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, m

train: Scanning /Users/adityagoyal/Desktop/ML internship/yolo/Bike Helmet Detection.v2-more-preprocessing-augmentation.yolov8 2/train/labels... 3546 images, 129 backgrounds, 0 corrupt: 100%|██████████| 3546/3546 [00:00<00:00, 4875.36it/s]


train: New cache created: /Users/adityagoyal/Desktop/ML internship/yolo/Bike Helmet Detection.v2-more-preprocessing-augmentation.yolov8 2/train/labels.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 131.4±33.3 MB/s, size: 30.3 KB)


val: Scanning /Users/adityagoyal/Desktop/ML internship/yolo/Bike Helmet Detection.v2-more-preprocessing-augmentation.yolov8 2/valid/labels... 126 images, 0 backgrounds, 0 corrupt: 100%|██████████| 126/126 [00:00<00:00, 4156.89it/s]

val: New cache created: /Users/adityagoyal/Desktop/ML internship/yolo/Bike Helmet Detection.v2-more-preprocessing-augmentation.yolov8 2/valid/labels.cache


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/222 [00:00<?, ?it/s]

In [3]:
def validate(frequency,vec):
    if(vec[0]>0) or (vec[1]>0):
        angle = vec[1]/math.sqrt(vec[0]**2 + vec[1]**2)
        projection = vec[1]
        frequency = frequency
        if(angle>0.3 and projection>60 and frequency>30):
            return True
    return False 

In [4]:
def compute_persons_line(result,final_id_line,line_coordinates):
    boxes = result.boxes 
    box_coordinates = boxes.xywh
    labels = result.boxes.cls
    x1,y1,x2,y2 = line_coordinates

    if not((boxes.id == None) and  not(labels==None)):
        for index in range(len(boxes.id)):
            id = boxes.id[index]
            label = labels[index]

            if(label==0):
                curr_x = box_coordinates[index][0]
                curr_y = box_coordinates[index][1]

                if(curr_x>x1 and curr_x<x2) and (curr_y<y1+15 and curr_y>y1-15 ):
                    final_id_line.add(id.item())

In [5]:
def compute_persons_trajectory(result,freq_dic,center_dic,vec_dic,current_id_set,final_id):
    labels = result.boxes.cls
    boxes = result.boxes 
    box_coordinates = boxes.xywh
    box_id = boxes.id
    box_cls = boxes.cls
    boxes_confidence = boxes.conf
    ids_frame = []
    
    if not((boxes.id == None) and  not(labels==None)):
        for index in range(len(boxes.id)):

            id = boxes.id[index]
            label = labels[index]

            if(label==0):

                ids_frame.append(id.item())
                current_id_set.add(id.item())

                freq_dic[id.item()] +=1 
                curr_x = box_coordinates[index][0]
                curr_y = box_coordinates[index][1]

                if(freq_dic[id.item()]<100):

                    if(id.item() in center_dic):
                        prev_x = center_dic[id.item()][0]
                        prev_y = center_dic[id.item()][1]
                        
                    else:
                        prev_x = 0
                        prev_y = 0

                    center_dic[id.item()] = [curr_x,curr_y]

                    if(id.item() in vec_dic):
                        vector = vec_dic[id.item()]
                        vec_x = vector[0]
                        vec_y = vector[1]
                        vec_dic[id.item()] = [vec_x + curr_x-prev_x, vec_y + curr_y-prev_y]
                    else:
                        vec_dic[id.item()] = [0,0]   

    for id in current_id_set.copy():
        if id not in ids_frame:
            if(validate(freq_dic[id],vec_dic[id])):
                final_id.add(id)
            current_id_set.remove(id)
        
    for id in ids_frame:
        current_id_set.add(id)

In [6]:
currentTime = datetime(2025,7,30,12,15,0)

In [7]:
# Example: generate 100 dummy frames (e.g., black images)
frame_height = 1080
frame_width = 1920
fps = fps
output_path = 'output_video_1.mp4'

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for .avi
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

In [8]:
""" Process a video stream frame-by-frame using YOLOv11 object tracking,
persistently tracking objects across frames and collecting results."""

frames = []

freq_dic = defaultdict(int)
center_dic = {}
vec_dic = {}
current_id_set = set()
final_id_trajectory = set()
final_id_line = set()

line_coordinates = [780, 400, 990, 400]
first_person_frame = None 

frame_no = 0
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:

        # Run YOLO11 tracking on the frame, persisting tracks between frames
        result = model.track(frame, persist=True,conf=0.45,save=True,show_labels=True)[0]
        frames.append(result)
        
        compute_persons_trajectory(result,freq_dic,center_dic,vec_dic,current_id_set,final_id_trajectory)
        person_count_trajectory = len(final_id_trajectory)
        if(len(final_id_trajectory)==1) and (not first_person_frame):
            first_person_frame = frame_no

        compute_persons_line(result,final_id_line,line_coordinates)
        person_count_line = len(final_id_line)

        # Display the current person count
        cv2.putText(frame, f"Persons: {person_count_trajectory}", (10, 1000), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        if (first_person_frame):
            currentTimeUpdated = currentTime + timedelta( seconds= (first_person_frame/total_frames)*seconds_of_video)
            date_time_string = currentTimeUpdated.strftime("%H:%M:%S")
            cv2.putText(frame, f"Time of Branch opening: {date_time_string}", (10, 1050), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame, f"Time of Branch opening: None", (10, 1050), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        out.write(frame)
        
        # Display the frame
        cv2.imshow("Person Counter", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    else:
        break

    frame_no += 1

out.release()
print("Video saved successfully.")

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 clock, 37.9ms
Speed: 2.1ms preprocess, 37.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 1 clock, 33.1ms
Speed: 1.4ms preprocess, 33.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 1 clock, 33.1ms
Speed: 1.7ms preprocess, 33.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 1 clock, 34.8ms
Speed: 1.6ms preprocess, 34.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 1 clock, 34.7ms
Speed: 1.2ms preprocess, 34.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 1 clock, 31.8ms
Speed: 1.2ms preprocess, 31.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track4

0: 384x640 1 clock, 31.6ms
Speed:

In [ ]:
final_vec_dic = {}
final_freq_dic = {}
for index in freq_dic:
    if(freq_dic[index]>=30):
        final_vec_dic[index] = vec_dic[index]
        final_freq_dic[index] = freq_dic[index]

In [ ]:
angle_dic = {}
projection_dic = {}
for idx in final_vec_dic:
    vec = final_vec_dic[idx]
    angle = vec[1]/math.sqrt(vec[0]**2 + vec[1]**2)
    angle_dic[idx] = angle
    projection_dic[idx] = vec[1]

In [ ]:
# keeping track of ids and their frequenxy 
# compute trajectory vector 
# angle between trajection vector and 90 degree vector 


# shubhams reccomnedations 
# explore approach of defining line and then seeing the intersection of person boundign box and line coordinate
# change confidence score 
# test v8 instead of v11

# need camera fixed for recommended approach 
# if not fixed, explore how to define your own line

# fientunign recommendations 
# datesets from roboflow 
# https://docs.ultralytics.com/modes/train/#usage-examples
# imagen files, with their annotation, bounding boxes in text files


# nfr -cctv gets stored in that 
# video analytics of manufacturing 
# how to make it real time (can make it but counter would icnrease after yolo stops detecting person)

In [ ]:
""""Detect number of people in a "huddle" based on their proximity.
For all bounding boxes of a specific label (e.g., "person"),
compute pairwise distances and identify if people are clustered.
If the number of people is less than or equal to the average
expected in the branch, ignore (return infinity = no huddle).
Returns: average distance between people, and actual count."""
def evaluate_overlap(bounding_boxes,labels,label,average_no_of_employees_present_in_branch):

    arr = []
    for box_idx in range(len(bounding_boxes)):
        if(labels[box_idx].item()==label):
            arr.append(bounding_boxes[box_idx].tolist()[:2])

    total_dist = 0
    for i in range(len(arr)):
        for j in range(i+1,len(arr)):
            total_dist += euclidean(arr[i],arr[j])


    average_no_of_employees_present_in_branch = 0
    if(len(arr)<= average_no_of_employees_present_in_branch): 
        total_dist=np.inf
        return total_dist,len(arr)
            
    return total_dist/len(arr),len(arr)

In [ ]:
"""
Iterate through detection results and:
- Track all unique person IDs across frames
- Find the frame with the tightest (smallest) person cluster (potential huddle)
- Count number of people in that huddle
- Also count how many are wearing ties (label 27)
"""

# huddles, no of employees, no_of_employyes wearing tie
min_dist = math.inf
min_r = None 
min_len = None
min_frame = None
no_of_ties = None

frame = 0
for r in frames:
    
    labels = r.boxes.cls
    boxes = r.boxes 
    box_coordinates = boxes.xywh

    dist,number = evaluate_overlap(box_coordinates,labels,0,8)
    if(dist<min_dist):
        min_r = r
        min_len = number  
        min_frame = frame   

        no_of_ties = 0
        for label in labels:
            if(label.item()==27):
                no_of_ties+=1
    
    frame+=1

In [ ]:
print("Results: ")
print("\n")
print(f"No of customers coming in branch: {count}")
print(f"Time of branch opening: {(first_person_frame/total_frames)*seconds_of_video} sec")  # need to see data
print(f"No of customer enetering with helmet: ")
print("\n")
print("Assuming we get cctv footage of a huddle happening,")
print(f"No of employees in huddle: {min_len}")
print(f"Time of huddle(frame): {(min_frame/total_frames)*seconds_of_video} sec")
print(f"No of employees wearing tie: {no_of_ties}")